In [13]:
!pip install nltk

In [14]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
import tensorflow as tf
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.preprocessing import text

from sklearn import metrics

In [15]:
w2v_model=KeyedVectors.load('tweets_cbow_300')


In [16]:
all_data = pd.read_csv('all.dev.features.csv')
all_data = all_data.dropna()
words = all_data['Column1']
labels = all_data['Column37']

d = {'Words': words, 'Labels': labels}

df = pd.DataFrame(data=d)
df = df.dropna()
df

,Words,Labels
0,الذرة,O
1,هي,O
2,أصغر,O
3,جزء,O
4,من,O
...,...,...
21908,BSD,I-MIS
21909,في,O
21910,طور,O
21911,التصميم,O


In [8]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ؟''' + string.punctuation
for i in range(len(punctuations)):
    df.drop((df.loc[df['Words'] == punctuations[i], ['Words', 'Labels']].index), axis=0, inplace=True)
    
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
for i in range(len(numbers)):
    df.drop((df.loc[df['Words'] == numbers[i], ['Words', 'Labels']].index), axis=0, inplace=True)


df

,Words,Labels
0,الذرة,O
1,هي,O
2,أصغر,O
3,جزء,O
4,من,O
...,...,...
21906,GNU,I-MIS
21908,BSD,I-MIS
21909,في,O
21910,طور,O


In [19]:
for word in df['Words']:
    if word in w2v_model.wv.vocab:
        continue
    
    else:
        df.drop((df.loc[df['Words'] == word ,['Words' ,'Labels']].index),axis=0,inplace=True)

df

,Words,Labels
1,هي,O
3,جزء,O
4,من,O
5,العنصر,O
6,الكيميائي,O
...,...,...
21903,تشغيل,I-MIS
21904,باسم,O
21909,في,O
21910,طور,O


In [20]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df['Labels'])
len(np.unique(encoded_labels)), encoded_labels

(9, array([8, 8, 8, ..., 8, 8, 8]))

In [21]:
y = tf.keras.utils.to_categorical(encoded_labels)

y.shape, y[0]

((11896, 9), array([0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32))

In [22]:
x = np.array(df['Words'])
x

array(['هي', 'جزء', 'من', ..., 'في', 'طور', 'التصميم'], dtype=object)

In [29]:
temp_sentences = []
temp_labels_sentences = []
temp = []

k=0
i=0
while(i < len(x)):
    sentence = ""
    for j in range(4):
        sentence += x[k]+' '
        temp.append(y[k])
        k+=1
        
    
    sentence = sentence.split(" ")
    sentence.pop()
    temp_sentences.append(sentence)
    temp_labels_sentences.append(temp)
    temp = []
    
    i+=4

labels_model=np.array(temp_labels_sentences)     
len(temp_sentences), len(temp_labels_sentences), len(temp_labels_sentences[0]), labels_model.shape

(2974, 2974, 4, (2974, 4, 9))

In [30]:
        
labels_model

array([[[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)

In [31]:
num_words = 11896
tokenizer=Tokenizer(num_words, lower=True)
tokenizer.fit_on_texts(df['Words'])

In [32]:
tokens=tokenizer.texts_to_sequences(temp_sentences)
tokens_pad=pad_sequences(tokens,maxlen=4,padding='post')
tokens_pad

array([[  32,  256,    2,  165],
       [ 409,   10, 1354,  257],
       [ 165,  410, 1355,  116],
       ...,
       [3792,  202, 3793,  462],
       [3794, 3795,    8,   29],
       [  83,    1, 1353,  405]])

In [33]:
vector_size = 300
gensim_weight_matrix = np.zeros((num_words ,vector_size))
gensim_weight_matrix.shape

(11896, 300)

In [34]:
for word, index in tokenizer.word_index.items():
    if index < num_words: # since index starts with zero 
        if word in w2v_model.wv.vocab:
            gensim_weight_matrix[index] = w2v_model[word]
        else:
            gensim_weight_matrix[index] = np.zeros(300)

C:\Users\sika\AppData\Local\Temp\ipykernel_24620\1213969340.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  gensim_weight_matrix[index] = w2v_model[word]


In [89]:
x_train,x_test,y_train,y_test=train_test_split(tokens_pad,labels_model,test_size=0.1,shuffle='true')

x_train.shape, y_train.shape

((2676, 4), (2676, 4, 9))

In [90]:
encoder_input_layer = tf.keras.layers.Input(shape=(4, ))
embedding_layer=tf.keras.layers.Embedding(input_dim = num_words, output_dim = 300,input_length= x_train.shape[1],weights = [gensim_weight_matrix],trainable = False)
encoder_embedding = embedding_layer(encoder_input_layer)


In [98]:
encoder_LSTM = LSTM(20,return_sequences=True)    
encoder_outputs= encoder_LSTM(encoder_embedding)

In [99]:
decoder_LSTM = LSTM(20,  return_sequences=True)
decoder_outputs = decoder_LSTM(encoder_outputs)

In [102]:

outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(9, activation='softmax'))(decoder_outputs)
model = tf.keras.Model(encoder_input_layer, outputs)

model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_31 (InputLayer)       [(None, 4)]               0         
                                                                 
 embedding_15 (Embedding)    (None, 4, 300)            3568800   
                                                                 
 lstm_26 (LSTM)              (None, 4, 20)             25680     
                                                                 
 lstm_27 (LSTM)              (None, 4, 20)             3280      
                                                                 
 time_distributed_11 (TimeDi  (None, 4, 9)             189       
 stributed)                                                      
                                                                 
Total params: 3,597,949
Trainable params: 29,149
Non-trainable params: 3,568,800
___________________________________________

In [105]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = 'accuracy')

In [106]:
history_embedding = model.fit(x_train, y_train, 
                                epochs = 25, batch_size = 120, 
                                verbose = 1 )

Epoch 1/25
23/23 [==============================] - 12s 15ms/step - loss: 1.9214 - accuracy: 0.6830
Epoch 2/25
23/23 [==============================] - 0s 13ms/step - loss: 1.1932 - accuracy: 0.8664
Epoch 3/25
23/23 [==============================] - 0s 12ms/step - loss: 0.8372 - accuracy: 0.8672
Epoch 4/25
23/23 [==============================] - 0s 12ms/step - loss: 0.7313 - accuracy: 0.8672
Epoch 5/25
23/23 [==============================] - 0s 12ms/step - loss: 0.6477 - accuracy: 0.8682
Epoch 6/25
23/23 [==============================] - 0s 11ms/step - loss: 0.5772 - accuracy: 0.8690
Epoch 7/25
23/23 [==============================] - 0s 11ms/step - loss: 0.5213 - accuracy: 0.8682
Epoch 8/25
23/23 [==============================] - 0s 11ms/step - loss: 0.4800 - accuracy: 0.8686
Epoch 9/25
23/23 [==============================] - 0s 12ms/step - loss: 0.4506 - accuracy: 0.8691
Epoch 10/25
23/23 [==============================] - 0s 12ms/step - loss: 0.4273 - accuracy: 0.8702
Epoch 11

In [110]:
y_pred = model.predict(x_test)


y_pred.shape,y_test.shape
#print(metrics.classification_report(y_pred, y_true))

((298, 4, 9), (298, 4, 9))

In [111]:
y_pred[0]

array([[9.69013257e-04, 1.85336161e-03, 3.28885915e-04, 9.25830565e-04,
        2.91307573e-04, 1.02780829e-03, 3.11822456e-04, 1.93719217e-03,
        9.92354751e-01],
       [8.15874431e-03, 1.60508707e-01, 5.17567946e-03, 1.12374080e-02,
        4.44295537e-03, 3.30830850e-02, 7.72425300e-03, 1.16853295e-02,
        7.57983863e-01],
       [2.15177672e-04, 1.43268437e-03, 7.04491176e-05, 2.31437851e-04,
        5.75684680e-05, 4.80296876e-04, 8.35323590e-05, 4.91732033e-04,
        9.96937156e-01],
       [7.84542717e-05, 3.35533696e-04, 1.91190811e-05, 7.33543566e-05,
        1.51407166e-05, 1.18149430e-04, 2.01056100e-05, 1.84154123e-04,
        9.99155998e-01]], dtype=float32)